In [ ]:
%matplotlib inline
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import ottawa_libraries as ol

In [ ]:
# open and plot the data. looks like Toronto
census_file = 'data/census_tracts.shp'
canada_census = gpd.read_file(census_file)
ottawa_census = canada_census[(canada_census.CMANAME=="Ottawa - Gatineau (Ontario part / partie de l'Ontario)")].copy()

In [ ]:
pop_data = pd.read_csv('data/98-401-X2016043_eng_CSV/98-401-X2016043_English_CSV_data.csv')
ottawa_census_data = pop_data[pop_data['GEO_CODE (POR)'].isin(ottawa_census.CTUID)]

In [ ]:
ottawa_area_data = ottawa_census_data.loc[ottawa_census_data["DIM: Profile of Census Tracts (2247)"]=='Land area in square kilometres'].copy()
ottawa_area_data['area'] = ottawa_area_data["Dim: Sex (3): Member ID: [1]: Total - Sex"]
ottawa_area_data = ottawa_area_data[['GEO_CODE (POR)', 'area']]

In [ ]:
ottawa_pop_data = ottawa_census_data.loc[ottawa_census_data["DIM: Profile of Census Tracts (2247)"]=='Population, 2016'].copy()
ottawa_pop_data['population'] = ottawa_pop_data["Dim: Sex (3): Member ID: [1]: Total - Sex"]
ottawa_pop_data = ottawa_pop_data[['GEO_CODE (POR)', 'population']]

In [ ]:
ottawa_density_data = ottawa_census_data.loc[ottawa_census_data["DIM: Profile of Census Tracts (2247)"]=='Population density per square kilometre'].copy()
ottawa_density_data['density'] = ottawa_density_data["Dim: Sex (3): Member ID: [1]: Total - Sex"]
ottawa_density_data = ottawa_density_data[['GEO_CODE (POR)', 'density']]

In [ ]:
ottawa_joined = ottawa_area_data.merge(ottawa_pop_data).merge(ottawa_density_data)

In [ ]:
ottawa_joined['population'] = pd.to_numeric(ottawa_joined['population'])
ottawa_joined['area'] = pd.to_numeric(ottawa_joined['area'])
ottawa_joined['density'] = pd.to_numeric(ottawa_joined['density'])
ottawa_joined['new_density'] = ottawa_joined['population'] / ottawa_joined['area']

In [ ]:
ottawa_joined['rel_dif'] = (ottawa_joined['density'] - ottawa_joined['new_density']) / ottawa_joined['density']

In [ ]:
# was expecting this to be even closer than it is but the peak difference is <2 %
ottawa_joined.rel_dif.describe()

In [ ]:
ottawa_joined.head()

#### Can we understand the issue with the projections?
So it turns out the world of projections is super complicated. I found [this](https://gis.stackexchange.com/questions/372564/userwarning-when-trying-to-get-centroid-from-a-polygon-geopandas) article super useful.

In [ ]:
ottawa_census['CTUID'] = pd.to_numeric(ottawa_census['CTUID'])

In [ ]:
ottawa_census['calc_area'] = ottawa_census.area/1000/1000

In [ ]:
ottawa_census = ottawa_census.merge(ottawa_joined, left_on='CTUID', right_on='GEO_CODE (POR)')

In [ ]:
# now much closer and the units actually make sense
ottawa_census.loc[ottawa_census.CTUID == 5050038, ['calc_area', 'area']]

In [ ]:
ottawa_census = ottawa_census.to_crs('epsg:4326')

In [ ]:
ottawa_census.crs

In [ ]:
# it appears to be angry because the units are in degrees
ottawa_census.area

In [ ]:
ottawa_census = ottawa_census.to_crs('+proj=cea')

In [ ]:
ottawa_census.crs

In [ ]:
ottawa_census['calc_area2'] = ottawa_census.area/1000/1000

In [ ]:
# lines up really nicely now and actually gives an answer closer to the one I'd measured.
ottawa_census.loc[ottawa_census.CTUID == 5050038, ['calc_area', 'area', 'calc_area2']]